# Play back Spot Dance Synced with Music

One you've developed some motion for Spot, it's time to put your dance together. Perhaps you want to add some music? Maybe a second robot? With the SyncedPerformanceCoordinator you can easily prepare and start multiple synchronized modalities

In [1]:
from spot_choreo_utils.choreo_playback.synced_audio_player import SyncedAudioPlayer
from spot_choreo_utils.choreo_playback.synced_performance_coordinator import SyncedPerformanceCoordinator, SyncedPeroformanceConfig
from spot_choreo_utils.choreo_playback.synced_spot_dancer import SyncedSpotDancer

# Setup Spot

In [ ]:
import logging
from spot_wrapper.wrapper import SpotWrapper

hostname="__ROBOT_IP__"
robot_name="__ROBOT_NAME__"
username="__USERNAME__"
password="__PASSWORD__"
has_arm = True
port = 0
logger = logging.Logger(name="spot_wrapper_logger")


spot_wrapper = SpotWrapper(
    username=username,
    password=password,
    hostname=hostname,
    robot_name=robot_name,
    logger=logger,
    use_take_lease=True,
    port=port,
)

spot_one = SyncedSpotDancer(spot_wrapper=spot_wrapper)

The SyncedSpotDancer class connects to the robot and handles dance playback. Set a single animation or full dance sequence to play on the robot.

In [ ]:
from spot_choreo_utils.choreo_creation.choreo_builders.animation_builder import (
    AnimationBuilder,
)
from spot_choreo_utils.serialization.serialization_utils import load_animation, load_sequence, save_animation
from spot_choreo_utils.paths import get_example_choreo_path
from pathlib import Path


# Play Animation Example
dance_path = Path(get_example_choreo_path(), "animations", "pose_to_pose_animation.pbtxt")
animation = load_animation(dance_path)
builder = AnimationBuilder.from_animation(animation)
spot_one.set_animation(builder)

# Play Sequence Example
#sequence_path = Path(get_example_choreo_path(), "sequences", "pose_to_pose_sequence.pbtxt")
#sequence = load_sequence(sequence_path)
#spot_one.set_sequence(sequence)

# Setup audio and dance

Load in some audio. By default we have a pretty uninspiring 60BPM tick, but feel free to swap it out with your own favorite song.

In [ ]:
from pathlib import Path
from spot_choreo_utils.paths import get_example_choreo_path

audio_file = Path(get_example_choreo_path(), "music", "60_bpm.wav")
print(audio_file)
audio_player = SyncedAudioPlayer(audio_file)

# Setup Playback
Add all of your performers/modalities to the coordinators

In [5]:
coordinator = SyncedPerformanceCoordinator()
coordinator.add_modality(spot_one)
coordinator.add_modality(audio_player)

# Play
Clear the dance floor, and away we go!

In [ ]:
config = SyncedPeroformanceConfig()
config.start_time_s = 0
config.end_time_s = 100
config.music_offset_s = 4
config.setup_timeout = 3
delay_once_ready = 2


await coordinator.perform_when_ready(config, delay_once_ready=delay_once_ready)